In [1]:
import numpy as np
import cv2
from imutils import face_utils
import dlib
from keras.models import Model, load_model
import winsound
from pygame import mixer
mixer.init()
sound = mixer.Sound('clock03.wav')

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
img_dim=(34,26)

pygame 2.0.2 (SDL 2.0.16, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def crop_eyes(img,points):
    x1, y1 = np.amin(points, axis=0)
    x2, y2 = np.amax(points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
    
    w = (x2 - x1) * 1.2
    h = w * img_dim[1] / img_dim[0]

    dx, dy = w / 2, h / 2
    
    min_x, min_y = int(cx - dx), int(cy - dy)
    max_x, max_y = int(cx + dx), int(cy + dy)
    eye_img=img[min_y:max_y,min_x:max_x] # input model
    rect_cord=[min_x,min_y,max_x,max_y]  # display eye
    
    return eye_img,rect_cord

    

In [3]:
model=load_model('BlinkModel.h5')


In [6]:
cap = cv2.VideoCapture(0)
counter=0
while True:
    
    ret, img = cap.read()
    if not ret:
        continue
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)
        lefteye,leftRect=crop_eyes(gray, shapes[36:42])
        righteye,rightRect=crop_eyes(gray, shapes[42:48])

        righteye=cv2.flip(righteye, flipCode=1)
        
        lefteye=cv2.resize(lefteye,img_dim,interpolation= cv2.INTER_LINEAR)
        righteye=cv2.resize(righteye,img_dim,interpolation= cv2.INTER_LINEAR)
        
        lefteye=lefteye.reshape(1, img_dim[1], img_dim[0], 1)/255.
        righteye=righteye.reshape(1, img_dim[1], img_dim[0], 1)/255.

    
        left_pred=model(lefteye)
        right_pred=model(righteye)

        state_l = 'Close' if left_pred <.1 else 'Open'
        state_r = 'Close' if right_pred <.1 else 'Open'


        
        cv2.rectangle(img, (leftRect[0],leftRect[1]), (leftRect[2],leftRect[3]), (0,100,255), 2)
        cv2.rectangle(img, (rightRect[0],rightRect[1]), (rightRect[2],rightRect[3]), (0,100,255), 2)
        
        if state_l =='Close' and state_r=='Close':
            counter+=1
            if counter==20:
                sound.play()
#                 sound.play(-1)
#                 k=cv2.waitKey()
#                 if k==115:
#                     sound.stop()
        else:
            counter=0
        
            
            
        if state_l=='Close':
            cv2.putText(img, 'Close', tuple(leftRect[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        else:
            cv2.putText(img, 'Open', tuple(leftRect[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            
        if state_r=='Close':
            cv2.putText(img, 'Close', tuple(rightRect[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        else:
            cv2.putText(img, 'Open', tuple(rightRect[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    
        
        

        
        
    cv2.imshow('img',img)
    if cv2.waitKey(30)==13:
        break
cap.release()
cv2.destroyAllWindows()